In [4]:
# https://data.tempe.gov/datasets/tempegov::1-08-crash-data-report-detail/about
# https://gis.tempe.gov/design/data-dictionary/1.08%20Crash%20Data%20(detail)/

import datetime as dt
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

print('Data preparation')
df = pd.read_csv("../dataset/crashdata.csv")
print('Columns:')
print(list(df.columns))

# merge similar severity levels and remove 'suspected' rows
df.loc[df["Injuryseverity"]=="Suspected Minor Injury", "Injuryseverity"] = "Possible Injury"
df = df[~df["Injuryseverity"].isin(['Suspected Minor Injury', 'Suspected Serious Injury'])]

# convert date column to datetime
df["DateTime"] = pd.to_datetime(df["DateTime"])
df["DateTime"] = df[["DateTime"]]-pd.Timedelta(hours=7)
df["hour"] = df["DateTime"].dt.hour
df["dayofweek"] = df["DateTime"].dt.dayofweek

# junction relation types
df.loc[df["JunctionRelation"].isin(
    ['Intersection 201', 'Intersection 4 Way 202', 'Intersection Interchange', 'Intersection Non Interchange', 
     'Intersection Other 204', 'Intersection Related 206', 'Intersection Related Interchange', 'Intersection Related Non Interchange',
     'Intersection T Inter 203', '117', '16', '201', '202', '203', '204', '205', '206', '255', '97']), "JunctionRelation"] = "Intersection"
df.loc[df["JunctionRelation"].isin(
    ['Unknown', 'Unknown Non Interchange']), "JunctionRelation"] = "Unknown"
df.loc[df["JunctionRelation"].isin(
    ['Not Junction Related']), "JunctionRelation"] = "Non intersection"
df.loc[~df["JunctionRelation"].isin(
    ['Not Junction Related', 'Intersection', 'Unknown', 'Unknown Non Interchange']), "JunctionRelation"] = "Non intersection"

# making a turn
df.loc[df["Unitaction_One"].isin(
    ['Making Left Turn', 'Making Right Turn', 'Making U Turn']), "Unitaction_One"] = "Making a Turn"

print('total obs.', len(df))

Data preparation
Columns:
['X', 'Y', 'OBJECTID', 'Incidentid', 'DateTime', 'Year', 'StreetName', 'CrossStreet', 'Distance', 'JunctionRelation', 'Totalinjuries', 'Totalfatalities', 'Injuryseverity', 'Collisionmanner', 'Lightcondition', 'Weather', 'SurfaceCondition', 'Unittype_One', 'Age_Drv1', 'Gender_Drv1', 'Traveldirection_One', 'Unitaction_One', 'Violation1_Drv1', 'AlcoholUse_Drv1', 'DrugUse_Drv1', 'Unittype_Two', 'Age_Drv2', 'Gender_Drv2', 'Traveldirection_Two', 'Unitaction_Two', 'Violation1_Drv2', 'AlcoholUse_Drv2', 'DrugUse_Drv2', 'Latitude', 'Longitude']
total obs. 39793


In [5]:
print("Generating the cleaned data")

df_cleaned = pd.DataFrame()

df_cleaned["incidentid"] = df["Incidentid"]
df_cleaned["severity"] = 0

df_cleaned.loc[df["Injuryseverity"]=="No Injury", "severity"] = 0
df_cleaned.loc[df["Injuryseverity"]=="Possible Injury", "severity"] = 1
df_cleaned.loc[df["Injuryseverity"]=="Non Incapacitating Injury", "severity"] = 2
df_cleaned.loc[df["Injuryseverity"]=="Incapacitating Injury", "severity"] = 3
df_cleaned.loc[df["Injuryseverity"]=="Fatal", "severity"] = 4

df_cleaned.loc[:, "age"] = df["Age_Drv1"].fillna(0).astype(int)*((df["Age_Drv1"]<80) & (df["Age_Drv1"]>15))
df_cleaned.loc[:, "total_injuries"] = df[["Totalinjuries"]]
df_cleaned.loc[:, "alcohol"] = 1*(df[["AlcoholUse_Drv1"]]=="Alcohol")

df_cleaned.loc[:, "hour_morning"] = 1*((df[["hour"]]>=6) & (df[["hour"]]<12))
df_cleaned.loc[:, "hour_afternoon"] = 1*((df[["hour"]]>=12) & (df[["hour"]]<18))
df_cleaned.loc[:, "hour_night"] = 1*((df[["hour"]]>=18) & (df[["hour"]]<24))
df_cleaned.loc[:, "hour_latenight"] = 1*(df[["hour"]]<6)

df_cleaned.loc[:, "nonintersection"] = 1*(df[["JunctionRelation"]]=="Non intersection")

df_cleaned.loc[:, "light_daylight"] = 1*(df[["Lightcondition"]]=="Daylight")
df_cleaned.loc[:, "light_darklighted"] = 1*(df[["Lightcondition"]]=="Dark Lighted")
df_cleaned.loc[:, "light_darknotlighted"] = 1*(df[["Lightcondition"]]=="Dark Not Lighted")
df_cleaned.loc[:, "light_dawndusk"] = 1*((df[["Lightcondition"]]=="Dawn") | (df[["Lightcondition"]]=="Dusk"))

df_cleaned.loc[:, "meteo_cloudy"] = 1*(df[["Weather"]]=="Cloudy")
df_cleaned.loc[:, "meteo_rain"] = 1*(df[["Weather"]]=="Rain")
df_cleaned.loc[:, "surf_wet"] = 1*(df[["SurfaceCondition"]]=="Wet")

df_cleaned.loc[:, "type_cyclist"] = 1*(df[["Unittype_One"]]=="Pedalcyclist")
df_cleaned.loc[:, "type_pedestrian"] = 1*(df[["Unittype_One"]]=="Pedestrian")
df_cleaned.loc[:, "type_driverless"] = 1*(df[["Unittype_One"]]=="Driverless")

df_cleaned.loc[:, "action_turn"] = 1*(df[["Unitaction_One"]]=="Making a Turn")
df_cleaned.loc[:, "action_lanes"] = 1*(df[["Unitaction_One"]]=="Changing Lanes")
df_cleaned.loc[:, "action_straight"] = 1*(df[["Unitaction_One"]]=="Going Straight Ahead")
df_cleaned.loc[:, "action_slowing"] = 1*(df[["Unitaction_One"]]=="Slowing In Trafficway")

df_cleaned.loc[:, "cause_speeding"] = 1*(df[["Violation1_Drv1"]]=="Speed To Fast For Conditions")
df_cleaned.loc[:, "cause_yield"] = 1*(df[["Violation1_Drv1"]]=="Failed To Yield Right Of Way")
df_cleaned.loc[:, "cause_unsafe"] = 1*(df[["Violation1_Drv1"]]=="Unsafe Lane Change")
df_cleaned.loc[:, "cause_signal"] = 1*(df[["Violation1_Drv1"]]=="Disregarded Traffic Signal")
df_cleaned.loc[:, "cause_following"] = 1*(df[["Violation1_Drv1"]]=="Followed Too Closely")
df_cleaned.loc[:, "cause_distraction"] = 1*(df[["Violation1_Drv1"]]=="Inattention Distraction")
df_cleaned.loc[:, "cause_turn"] = 1*(df[["Violation1_Drv1"]]=="Made Improper Turn")

df_cleaned.loc[:, "acc_rearend"] = 1*(df[["Collisionmanner"]]=="Rear End")
df_cleaned.loc[:, "acc_leftturn"] = 1*(df[["Collisionmanner"]]=="Left Turn")
df_cleaned.loc[:, "acc_sides"] = 1*(df[["Collisionmanner"]]=="Sideswipe Same Direction")
df_cleaned.loc[:, "acc_angle"] = 1*((df[["Collisionmanner"]]=="ANGLE (Front To Side)(Other Than Left Turn)") | (df[["Collisionmanner"]]=="Angle - Other Than Left Turn 2"))

display(df_cleaned)

Generating the cleaned data


,incidentid,severity,age,total_injuries,alcohol,hour_morning,hour_afternoon,hour_night,hour_latenight,nonintersection,...,cause_yield,cause_unsafe,cause_signal,cause_following,cause_distraction,cause_turn,acc_rearend,acc_leftturn,acc_sides,acc_angle
0,2584487,0,0,0,0,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,1
1,2584442,2,26,3,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2589844,0,19,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
3,2579417,0,43,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,2584480,0,34,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39912,3729485,1,42,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
39913,3730560,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
39914,3729491,1,26,1,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
39915,3729497,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [31]:
# export the data as .csv with "\t" seperator
df_cleaned.to_csv("tempe_cleaneddata.csv", sep="\t", index=0)